<a href="https://colab.research.google.com/github/FredBethonico/GerarNPC_T20RPG_IA/blob/sqlite-atributesDB/GerarNPC_IA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerar NPC aleatório

Aplicação para auxiliar mestres de RPG do sistema Tormenta20 a criarem NPCs de forma aleatória e rápida.

Neste código, utiliza-se IA para gerar uma pequena apresentação contextualizada no mundo de Arton e em primeira pessoa do NPC recém gerado.

*Observação: Esse código está sendo escrito e aprimorado a medida que estudo Python. Comecei há poucos meses e sempre que aprendo algo novo que possa se encaixar neste projeto, faço a implementação, como fiz agora com a API do Gemini.*


### Listas dos atributos

Raças, Classes e Origens retiradas dos livros:
- Tormenta 20
- Ameaças de Arton

*Material original da Editora Jambô, reproduzido parcialmente para fins de entretenimento e sem nenhum interesse lucrativo.*

### Main

Parte central do código.
A ideia de fazer este código orientado a objetos é de que poderei implementar outras funcionalidades próprias de jogos de RPG no futuro reutilizando esse mesmo trecho de código.

In [116]:
# Pacotes importados

import random
import textwrap
import sqlite3 as sql

In [117]:
con = sql.connect("pools.db")
cur = con.cursor()

In [132]:
# Funções utilitárias

aleatorio = lambda pool: random.randint(0,(len(pool)-1))

def select (stringtabela):
  ''' Gerar consulta do banco de dados. Parâmetro: string com o nome da tabela. Retorno: lista com os valores da tabela. '''
  query = "SELECT * FROM " + stringtabela
  dados = cur.execute(query).fetchall()
  return [i[0] for i in dados] # `list comprehension` para transformar os dados em lista de strings

def aleatorizar (atributo):
  ''' Parâmetro: uma lista. Retorno: Elemento aleatório da lista. '''
  return atributo[aleatorio(atributo)]


In [133]:
# Armazenar dados das tabelas, como strings, em variáveis

# Nomes
nomesM = select("nomesM")
nomesF = select("nomesF")
sobrenomes = select("sobrenomes")
apelidosM = select("apelidosM")
apelidosF = select("apelidosF")

# Raças
racasM = select("racasM")
racasF = select("racasF")

# Sub-raças
golem_chassi = select("golem_chassi")
golem_energia = select("golem_energia")
subraca_osteon = select("subraca_osteon")
subraca_suraggel = select("subraca_suraggel")
subraca_trog = select("subraca_trog")
subraca_trovao = select("subraca_trovao")
subraca_moreau = select("subraca_moreau")

# Classes
classesM = select("classesM")
classesF = select("classesF")

# Origens
origensM = select("origensM")
origensF = select("origensF")

# Divindades
divindades = select("divindades")

In [120]:
# Função das sub-raças

def gerarSubRaca(raca):
  if raca == "Golem":
    raca1 = aleatorizar(golem_chassi)
    raca2 = aleatorizar(golem_energia)
    raca = raca + " de " + raca1 + " energizado por " + raca2
  elif raca == "Osteon":
    raca = aleatorizar(subraca_osteon)
  elif raca == "Suraggel":
    raca = aleatorizar(subraca_suraggel)
  elif raca == "Trog":
    raca = aleatorizar(subraca_trog)
  elif raca == "Povo Trovão":
    raca = aleatorizar(subraca_trovao)
  elif raca == "Moreau":
    raca = aleatorizar(subraca_moreau)
  return raca

In [121]:
# Classe NPC

class NPC():
  def __init__(self, nome, sobrenome, apelido, raca, classe, origem, divindade, genero):
    self.nome = nome
    self.sobrenome = sobrenome
    self.apelido = apelido
    self.raca = raca
    self.classe = classe
    self.origem = origem
    self.divindade = divindade
    self.genero = genero
  def GerarNPC(self, genero):
    ficha_npc = ""
    if genero == "M":
      self.nome = aleatorizar(nomesM)
      self.sobrenome = aleatorizar(sobrenomes)
      self.apelido = aleatorizar(apelidosM)
      self.raca = aleatorizar(racasM)
      self.raca = gerarSubRaca(self.raca)
      self.classe = aleatorizar(classesM)
      self.origem = aleatorizar(origensM)
      self.divindade = aleatorizar(divindades)
      self.genero = "M"
    elif genero == "F":
      self.nome = aleatorizar(nomesF)
      self.sobrenome = aleatorizar(sobrenomes)
      self.apelido = aleatorizar(apelidosF)
      self.raca = aleatorizar(racasF)
      self.raca = gerarSubRaca(self.raca)
      self.classe = aleatorizar(classesF)
      self.origem = aleatorizar(origensF)
      self.divindade = aleatorizar(divindades)
      self.genero = "F"
    else:
      print("Favor entrar com o parâmetro 'F' para geração de nome feminino, ou 'M' para nome masculino.")
  def verFicha(self):
    if self.genero == "M":
      return f'{self.nome} {self.sobrenome} {self.apelido}\n {self.raca}\n Classe: {self.classe}\n Origem: {self.origem}\n Devoto de {self.divindade}'
    elif self.genero == "F":
      return f'{self.nome} {self.sobrenome} {self.apelido}\n {self.raca}\n Classe: {self.classe}\n Origem: {self.origem}\n Devota de {self.divindade}'


In [122]:
# Função para criar um NPC

def novoNPC(genero):
    npc = NPC("","","","","","","","")
    npc.GerarNPC(genero)
    return npc

def promptNPC(npc):
  prompt = ""
  prompt = npc.verFicha()
  return prompt


### Resultado inicial da aplicação de Gerar NPC

A princípio o programa intencionava apenas gerar o objeto NPC e exibí-lo no console de forma organizada.
Agora transformei o resultado em uma string contínua para entrar como um comando no modelo IA.

In [134]:
# Gerar NPCs

npcTeste = novoNPC("F")
promptNPC(npcTeste)

'Lucrécia Yuuta A cega\n Golem de Bronze energizado por Elemental de Terra\n Classe: Bucaneira\n Origem: Mateira\n Devota de Tanna-Toh'

### Implementação da IA

Este trecho do código foi desenvolvido durante a Imersão IA 2ª Edição, da Alura!

Também utilizei a documentação disponibilizada pela Google e contei com o apoio do próprio Gemini para corrigir alguns bugs e tirar dúvidas sobre a implementação e uso da API.

UPDATE: Troquei o formato de geração do modelo. Ao invés de utilizar o *'chat'*, agora estou utilizando *'generate content'*. Percebi que torna a aplicação mais simples, rápida e confiável.
Portanto o modelo não retém histórico do que já criou, ele sempre se baseia nas instruções de contexto fornecidas.

In [136]:
# Instalação

!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 8.4 MB/s eta 0:00:00


In [137]:
# Importação e configuração do modelo
# Escolhas de configuração e segurança copiei do AI Studio, de quando simulei essa ideia por lá

import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

generation_config = {
  "temperature": 1.0,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 2500,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


### Informações de lore de Tormenta20

Aqui tive que preparar mais informações de lore do RPG, para que o modelo "dê vida" aos personagens de forma imersiva.

In [138]:
# IDEIA: Passar deuses e raças para variáveis específicas, criar condicionais para o modelo receber somente as variaveis condizentes com a ficha de NPC gerada.
# IDEIA: Fornecer contextos para as outras raças também
# Informações para alimentar o modelo

aharadak = "Aharadak: O deus da Tormenta. Originado por ela e seu maior campeão. Ascendeu à divindade após matar o antigo líder do panteão divino e deus da força Tauron (também deus dos minotauros). *Devotos deste deus obrigatoriamente são cultistas insanos ou malignos, podendo ser inclusive grandes vilões.*\n\n"
allihanna = "Allihanna: A deusa da natureza. Associada à pureza da natureza, à bondade e à vida natural. Criadora das Dallan e dos Moreau. *Devotos dessa deusa normalmente são bondosos, pregam a proteção da natureza e de seus seres vivos. Em alguns casos mais raros, podem ser militantes fanáticos que acabam causando problemas pela sua devoção extremista.*\n\n"
arsenal = "Arsenal: O deus da guerra. Já foi um grande vilão presente no mundo de Arton. Hoje seus sacerdotes levam o ideal da guerra pela guerra por toda Arton. *Devotos desse deus normalmente são malignos, pregam a guerra como salvação e possuem como missão espiritual espalhar conflitos armados pelo mundo. Existem exceções em que devotos de Arsenal possuem outras visões e comportamentos a respeito da guerra.*\n\n"
azgher = "Azgher: O deus do dia, o próprio Sol. Inimigo de Tenebra a deusa-lua. *Devotos de Azgher normalmente são justos e possuem como obrigação divina combater o mal e as mentiras, praticar a caridade e o altruísmo, sendo sempre gratos à luz solar.*\n\n"
hyninn = "Hyninn: O deus da trapaça, da enganação. Patrono dos ladinos e das criaturas espertas e sorrateiras. Também associado à diversão das travessuras. *Devotos de Hynnin podem ser bondosos ou malignos em mesma proporção. Podem ver a enganação como uma diversão, ou como um meio para chegar a um bom final; ou podem usá-la para cometer grandes crimes e causar grande mal ao mundo.*\n\n"
kallyadranoch = "Kallyadranoch: O deus do poder, criador dos dragões. *Devotos de Kallyadranoch, ou como gostam de chamá-lo 'Kally', podem ser bondosos ou malignos, mas quase sempre são vaidosos e orgulhosos, pois seu deus é o criador dos dragões e da potência elemental no mundo. Têm como missão buscar poder e se tornarem seres muito poderosos homenageando a seu deus.*\n\n"
khalmyr = "Khalmyr: O deus da justiça. Primeiro líder do panteão. *Devotos de Khalmyr pregam a justiça a qualquer custo, prezam sempre por uma leitura fria e que siga os princípios da lei (seja lá qual for a lei). Por isso costumam ser bondosos, mas não exclusivamente, já que juíses de Khalmyr podem, com frequência, tomar decisões que levam à ruína de alguém por se tratar de algo 'justo' na lei.*\n\n"
lena = "Lena: A deusa da vida e da fertilidade. Matrona das mães. *Devotos de Lena são bondosos. Pregam a vida, a fertilidade e a maternidade como dons maiores e sagrados. Focam seus esforços em ajudar aos outros, especialmente utilizando dos dons mágicos da cura que a deusa provê aos fiéis.*\n\n"
linwu = "Lin-Wu: O deus da honra. Originário de Tamu-Ra. Patrono dos samurais. *Devotos de Lin-Wu são honrados, buscam sempre seguir determinado código de honra que, se quebrado, estariam ofendendo ao próprio deus.*\n\n"
marah = "Marah: A deusa da paz, da arte e do amor. Seus devotos são pacifistas e procuram apreciar as coisas boas do mundo. *Devotos de Marah são sempre pacifistas, sempre buscam encerrar conflitos ou resolver situações através da paz. Também possuem como missão, espalhar o amor e a arte pelo mundo, trazendo alegria para todos os corações. E também aproveitando a vida o quanto podem.*\n\n"
megalokk = "Megalokk: O deus dos monstros, irmão de Alihanna. Associado ao lado selvagem e mortal da natureza. Criador de todos os monstros naturais de Arton. *Devotos normalmente são malignos, mas podem existir exceções. Seguidores de Megalokk veneram os monstros ou são monstros eles próprios. Têm por obrigação divina sempre dar voz à seus instintos e agir sempre com a velocidade da ação, recusando planos e pensamentos.*\n\n"
nimb = "Nimb: O deus do caos, da sorte e do azar. Um ditado popular de Arton diz que 'se Khalmyr coloca o tabuleiro, é Nimb quem move as peças'. *Devotos de Nimb pregam a loucura e o acaso, a sorte e o azar. Por conta disso muitas vezes não podem ser lidos como bons ou maus, eles apenas são.*\n\n"
oceano = "Oceano: O deus dos vastos mares e oceanos. Dizem que seu domínio é o maior de todo o mundo. Criador dos seres marinhos, incluindo sereias e tritões. Patrono de todos que vivem no/do mar. *Devotos de Oceano veneram ao mar e suas criaturas. Possuem como missão proteger os mares e manter a harmonia entre o oceano e o mundo seco.*\n\n"
sszzaas = "Sszzaas: O deus da traição. Já foi um dos grandes vilões do mundo de Arton. Patrono das serpentes e criador dos povos nagah e das medusas. *Devotos de Sszzaas são sempre malignos. Pregam tragédias e espalham mentiras pelo mundo, se infiltram em sociedades pacíficas para espalhar a desconfiança e o caos, preferem abordagens traiçoeiras à resoluçaõ aberta de problemas. Devotos desse deus podem ser inclusive grandes vilões. *\n\n"
tannatoh = "Tanna-Toh: A deusa do conhecimento e da verdade. Seus devotos espalham educação formal pelo mundo e possuem votos contra a mentira, jurando dizer sempre a verdade. *Devotos da deusa Tanna-Toh pregam o conhecimento e a sabedoria como virtudes e a civilização como único caminho possível. Possuem como missão levar a educação por onde andam e como devoção são obrigados a dividir os conhecimentos que possuem e nunca podem mentir, ou serão punidos por sua deusa.*\n\n"
tenebra = "Tenebra: A deusa da noite, a própria Lua. Inimiga de Azgher, o deus-sol. Matrona das criaturas noturnas e subterrâneas. Junto com Khalmyr, em um passado muito distante, criou os anões. *Devotos de Tenebra podem ser bondosos ou malignos, possuem como obrigação reverenciar a noite, a lua e as estrelas, proteger segredos e mistérios e reverenciar os mortos-vivos. Devem sempre rejeitar o sol e a luz.*\n\n"
thwor = "Thwor: O deus dos goblinoides. Responsável por unificar as diversas raças goblinoides sob um único ideal, o Duyshidakk. Essa espécie de 'orgulho de raça' goblinoide os levou aos píncaros da conquista e reconquistaram todo o continente sul de volta à sua antiga glória e liberdade. *Devotos de Thwor podem ser bondosos ou malignos e reverenciam a lealdade, a força e a coragem. Promovem a união dos povos goblinoides e a aliança entre outros povos quando possível. Também reverenciar a mutação social, a revolução.*\n\n"
thyatis = "Thyatis: O deus da ressureição e da profecia. Possui a forma de uma fênix. *Devotos de Thyatis quase sempre são bondosos. Possuindo o dom da profecia, viajam o mundo ajudando aos necessitados. Renegam a mentira e a morte, pois pregam a vida. Em fato, devotos desse deus muitas vezes são agraciados com a ressurreição ou a imortalidade. São obrigados a oferecer sempre o perdão, a clemência e a redenção.*\n\n"
valkaria = "Valkaria: A deusa da ambição, criadora da raça humana. Atual líder do panteão divino. Matrona dos aventureiros e dos exploradores, dos ambiciosos e dos inventivos. *Devotos de Valkaria podem ser bons ou maus. Pregam pela liberdade e pela mudança constante. Nunca ficam no mesmo lugar por muito tempo. Normalmente são otimistas e rebeldes, combatem a tirania e toda forma de imposição e controle social. Desfrutam a vida com ambição, paixão e coragem.*\n\n"
wynna = "Wynna: A deusa da magia, uma das maiores dádivas e potências de Arton. Criadora das Sílfides e dos Qareen. *Devotos de Wynna podem ser bondosos ou malignos. Reverenciam a magia arcana e seus praticantes, promovem o ensino e aprendizagem da magia e vêem a magia como ferramenta máxima para melhoria da vida de todos.*\n\n"

info_tormenta = "Tormenta: A tormenta é o maior mal que assola o mundo de Arton. É conhecida como a 'anti-criação' e é temida por todos as criaturas vivas. Suas propriedades aberrantes são extremamente tóxicas e sua simples visão leva seres vivos à loucura. Uma vez completamente consumido pela anti-criação, uma criatura se torna um Lefeu e passa a integrar o organismo coletivo que é a Tormenta.\n\n"
info_TamuRa = "Tamu-Ra: Tamu-Ra é uma ilha distante ao nordeste de Arton. Possui traços característicos do Japão feudal. A terra do deus da honra Lin-Wu. Nesta ilha existia o grandioso Império de Jade que foi devastado pela Tormenta, depois de uma batalha longa e trágica, um grupo de heróis formado por indivíduos de várias partes do mundo, com a ajuda do deus Lin-Wu, conseguiram derrotar a tempestade, porém as terras do Império de Jade ficaram maculadas para sempre pela anti-criação.\n\n"

lefou = "Lefou: O Lefou é um indivíduo tocado pela tormenta, mas que não sucumbiu à sua loucura. O Lefou é um ser de outra raça mas que nasceu com traços próprios da Tempestade Rubra (como também é chamada a tormenta) e por isso é estigmatizado pelas sociedades em geral.\n\n"
qareen = "Qareen: São seres extraplanares, conhecidos também como 'meio-gênios' por se aproximarem muito dos seres mágicos. São criações da deusa da magia Wynna.\n\n"
golem = "Golem: Seres artificiais, construídos.\n\n"
hynne = "Hynne: Seres baixinhos, com pés peludos (como hobbits ou halflings)\n\n"
kliren = "Kliren: Também chamados de meio-gnomos são seres altos e esguios provenientes de algum lugar do éter cósmico. Sua origem é misteriosa e podem facilmente serem vistos como humanos 'um pouco mais altos que o normal'.\n\n"
osteon = "Osteon: São indivíduos que voltaram à vida na forma de esqueletos sencientes. Seu surgimento é atribuido à morte do antigo deus da morte Ragnar.\n\n"
silfide = "Sílfide: São seres pequenininhos, com asas e aspécto feérico. Sim, são fadas.\n\n"
aggelus = "Aggelus: São indivíduos que possuem traços 'angelicais', advindos de uma ascendência de outros planos.\n\n"
sulfure = "Sulfure: São indivíduos que possuem traços 'demoníacos', advindos de uma ascendência de outros planos.\n\n"
trog = "Trog: São seres semelhantes a crocodilos antropomorfizados.\n\n"
tabrachi = "Tabrachi: Seres aquáticos, semelhantes a sapos antropomorfizados. Vivem em rios e lagos, em comunidades de outros da sua espécie.\n\n"
kallyanach = "Kallyanach: Seres semelhantes a humanos, porém com traços dracônicos (como asas, caldas e as vezes escamas), atribuídos à divindade Kallyadranoch.\n\n"
kaijin = "Kaijin: Raça típica de Tamu-Ra. Monstros criados pela Tempestade Rubra. Apesar da maioria dos Kaijin serem criaturas insanas e selvagens, alguns raros casos possuem consciência e agem contra sua natureza adotando caminhos de sua própria escolha.\n\n"
kappa = "Kappa: Raça típica de Tamu-Ra. Seres aquáticos tamuranianas (de Tamu-Ra), inspirados nos Yokai Kappa japoneses.\n\n"
mashin = "Mashin: Raça típica de Tamu-Ra. Similar ao Golem, porém com características tamuranianas, como uma aparência de uma armadura samurai\n\n"
nezumi = "Nezumi: Raça típica de Tamu-Ra. Seres baixos, semelhantes a ratos antropomorfizados.\n\n"
tengu = "Tengu: Raça típica de Tamu-Ra. Seres alados, com características de ave, similares aos Yokai Tengu japoneses.\n\n"
povotrovao = "Povo Trovão: Ceratops, Pteros, Velocis, Voracis. São seres semelhantes a dinossauros e animais pré-históricos antropomorfizados, equivalentes à Triceratops, pterodáctilo, antílope e dente-de-sabre.\n\n"
moreau = "Moreau: Povos das ilhas Moreania. Uma terra distante onde animais evoluíram em raças sencientes. Cada herança representa a forma do animal antropomorfizado.\n\n"


### Orientando o modelo

Com as informações prévias preparadas, aqui definimos as "*System Instructions*", sobre como o modelo deve responder. Em seguida utilizamos o sistema de "*few-shot learning*" para exemplificar ao modelo qual o formato padrão das respostas que queremos. Além do padrão de respostas, também queremos direcionar os detalhes das respostas para os dados do personagem gerado, então fornecemos informações específicas apenas sobre a ficha gerada aleatoriamente.

A importância de repetir o processo a cada vez que o programa for executado, é a garantia de que as respostas vão estar sempre dentro do padrão desejado.

In [150]:
# Ensinando o modelo
def main():
  genero = input("Insira M para gera um NPC com nomes e adjetivos masculinos.\nInsira F para gerar um NPC com nomes e adjetivos femininos.\n Digite: ")
  global npcZero
  npcZero = novoNPC(genero)
  promptInicial = promptNPC(npcZero)

  match npcZero.divindade:
    case "Aharadak":
      info_divindade = aharadak
    case "Allihanna":
      info_divindade = allihanna
    case "Arsenal":
      info_divindade = arsenal
    case "Azgher":
      info_divindade = azgher
    case "Hyninn":
      info_divindade = hyninn
    case "Kallyadranoch":
      info_divindade = kallyadranoch
    case "Khalmyr":
      info_divindade = khalmyr
    case "Lena":
      info_divindade = lena
    case "Lin-Wu":
      info_divindade = linwu
    case "Marah":
      info_divindade = marah
    case "Megalokk":
      info_divindade = megalokk
    case "Nimb":
      info_divindade = nimb
    case "Oceano":
      info_divindade = oceano
    case "Sszzaas":
      info_divindade = sszzaas
    case "Tanna-Toh":
      info_divindade = tannatoh
    case "Tenebra":
      info_divindade = tenebra
    case "Thwor":
      info_divindade = thwor
    case "Thyatis":
      info_divindade = thyatis
    case "Valkaria":
      info_divindade = valkaria
    case "Wynna":
      info_divindade = wynna

  match npcZero.raca:
    case "Lefou":
      info_raca = lefou
    case "Qareen":
      info_raca = qareen
    case "Hynne":
      info_raca = hynne
    case "Kliren":
      info_raca = kliren
    case "Osteon" | "Osteon soterrado":
      info_raca = osteon
    case "Sílfide":
      info_raca = silfide
    case "Aggelus":
      info_raca = aggelus
    case "Sulfure":
      info_raca = sulfure
    case "Trog" | "Trog anão":
      info_raca = trog
    case "Tabrachi":
      info_raca = tabrachi
    case "Kallyanach":
      info_raca = kallyanach
    case "Kaijin":
      info_raca = kaijin
    case "Kappa":
      info_raca = kappa
    case "Mashin":
      info_raca = mashin
    case "Nezumi":
      info_raca = nezumi
    case "Tengu":
      info_raca = tengu
    case "Povo Trovão" | "Ceratops" | "Pteros" | "Velocis" | "Voracis":
      info_raca = povotrovao
    case "Moreau" | "Moreau coruja" | "Moreau hiena" | "Moreau raposa" | "Moreau serpente" | "Moreau búfalo" | "Moreau coelho" | "Moreau crocodilo" | "Moreau gato" | "Moreau leão" | "Moreau lobo" | "Moreau morcego" | "Moreau urso" :
      info_raca = moreau
    case _:
      info_raca = golem

  prompt_parts = [
    "Estamos em um mundo fantástico de RPG no universo de Tormenta20. Você atuará conforme o papel de personagem que lhe for atribuído e fará uma breve apresentação.",
    "Segue informações importantes sobre o universo. Gentileza basear seu texto fortemente nas informações a seguir:",
    info_tormenta, info_divindade, info_raca,
    "Agora vou enviar EXEMPLOS: Responda com mensagens de saudação e apresentação em primeira pessoa, conforme exemplos a seguir:"
    "input: Bomir Dracomestre O azarado\n Trog\n Classe: Nobre\n Origem: Guarda\n Devoto de Valkaria\n\nEste personagem se chama \"Bomir Dracomestre\", cujo apelido é \"O azarado\" por algo que aconteceu em sua vida. Sua raça é \"Trog\" e sua classe é \"Nobre\". Sua origem(background) é \"Guarda\" e a divindade que segue é \"Valkaria\".",
    "output: Saudações, eu me chamo Bomir Dracomestre e sou um guarda real da cidade de Ventossol. No meu tempo livre gosto de caçar e pescar no pântano e aos finais de semana faço pequenas viagens em homenagem à deusa Valkaria.",
    "input: Jonah Montealto O azarado\n Osteon\n Classe: Bárbaro\n Origem: Fazendeiro\n Devoto de Thyatis\n\nEste personagem se chama \"Jonah Montealto\", cujo apelido é \"O justo\" por algo que aconteceu em sua vida. Sua raça é \"Osteon \" e sua classe é \"Bárbaro\". Sua origem(background) é \"Fazendeiro\" e a divindade que segue é \"Thyatis\".",
    "output: Olá. sou Jonah Montealto, sou fazendeiro em Proventus. Eu perdi tudo o que tinha em um jogo de cartas quando eu tinha tudo para ganhar, por isso me chamam de O azarado. Sou devoto de Thyatis pois acredito que todos merecem segundas chances.",
    "input: Valéria Pottala A robusta\n Dahllan\n Classe: Cavaleira\n Origem: Mateira\n Devota de Oceano\n\nEsta personagem se chama \"Valéria Pottala\", cujo apelido é \"A robusta\" por algo que aconteceu em sua vida. Sua raça é \"Dahllan\" e sua classe é \"Cavaleira\". Sua origem(background) é \"Capanga\" e a divindade que segue é \"Oceano\".",
    "output: Oi! Sou Valéria Pottala, sou uma filha da natureza que entrou para a ordem de cavaleiros do mar e hoje venero ao deus Oceano. Já fui capanga de um grande vilão druida de um bosque distante, mas abandonei tudo isso para buscar justiça em meio aos lobos do mar.",
    "input: Khaya Jarpentaz A abandonada\n Hynne\n Classe: Bárbara\n Origem: Capanga\n Devota de Hyninn\n\nEsta personagem se chama \"Khaya Jarpentaz\", cujo apelido é \"A abandonada\" por algo que aconteceu em sua vida. Sua raça é \"Hynne\" e sua classe é \"Bárbara\". Sua origem(background) é \"Capanga\" e a divindade que segue é \"Hyninn\".",
    "output: Oi! Sou Khaya Jarpentaz, nasci em uma grande cidade e fui abandonada ainda criança. Desde então me criei nas ruas, dentro do mundo do crime. Foi dessa forma que treinei para me tornar uma bárbara urbana e também foi nesse caminho que encontrei minha divindade Hyninn.",
    "input: Verona Arvenal A abençoada\n Moreau raposa\n Classe: Barda\n Origem: Capanga\n Devota de Azgher\n\nEsta personagem se chama \"Verona Arvenal\", cujo apelido é \"A abençoada\" por algo que aconteceu em sua vida. Sua raça é \"Moreau raposa\" e sua classe é \"Barda\". Sua origem(background) é \"Seguidora\" e a divindade que segue é \"Azgher\".",
    "output: Saudações a todos! Sou a barda Verona Arvenal! Abençoada por Azgher ainda no berço carrego a alcunha de 'abençoada' pelo meu povo. Quando criança era uma fiel devota e seguidora, quando aprendi a música, decidi viajar o mundo levando a palavra de Azgher a todos.",
    "input: Halsin Martrel O belo\n Lefou\n Classe: Arcanista\n Origem: Eremita\n Devoto de Arsenal\n\nEste personagem se chama \"Halsin Martrel\", cujo apelido é \"O belo\" por algo que aconteceu em sua vida. Sua raça é \"Lefou\" e sua classe é \"Arcanista\". Sua origem(background) é \"Eremita\" e a divindade que segue é \"Arsenal\".",
    "output: Olá, mortais. Meu nome é Halsin Martrel, o belo. Desde que tive o corpo marcado pela maldição da tormenta, passei a me chamar de belo para incomodar aos demais. Por ser devoto de Arsenal, viajo por Arton espalhando a guerra e a violência, pois acreditamos que apenas pela guerra este mundo será curado.",
    "input: ", promptInicial,
    "output: ",
  ]
  global resposta
  resposta = model.generate_content(prompt_parts)

  print("#" * 80)
  print("## Ficha do NPC".center(80))
  print("#" * 80)
  print(npcZero.verFicha().center(80))

  print("\n" + "*" * 80)
  print("## História do Personagem".center(80))
  print("*" * 80)
  print(textwrap.fill(resposta.text, width=80))

### Executar aplicação

Executando a aplicação pela função recém criada, vemos o retorno impresso no console. Se trata da ficha do NPC gerado aleatoriamente e a apresentação contextualizada no *lore* de tormenta que o Gemini escreveu, dando vida ao novo personagem.



In [151]:
if __name__ == "__main__":
    main()

Insira M para gera um NPC com nomes e adjetivos masculinos.
Insira F para gerar um NPC com nomes e adjetivos femininos.
 Digite: F
################################################################################
                                ## Ficha do NPC                                 
################################################################################
Myrna Xernoval A incorrigível
 Nagah
 Classe: Arcanista
 Origem: Fazendeira
 Devota de Tanna-Toh

********************************************************************************
                           ## História do Personagem                            
********************************************************************************
Olá a todos! Sou Myrna Xernoval, uma nagah a serviço da deusa Tanna-Toh. Costumo
ser chamada de "A Incorrigível" por alguns, por minha insistência em levar a
verdade à luz. Apesar de ter nascido no campo, sempre fui fascinada pelos
mistérios da magia e do conhecimento. A partir do mom

### Futuro do projeto

Este é um projeto pessoal que está longe de terminar. Na verdade eu apenas comecei.

À medida que eu avançar nos meus estudos de programação, pretendo aprimorar ainda mais este código.

Também pretendo criar outros programas que, junto com esse, serão ferramentas muito úteis para me auxiliar antes, durante e depois das sessões de RPG.

*E claro, tudo sempre ficará disponível no meu GitHub para quem mais quiser utilizar.*